# Text classification with BERT using TF Text

## Setup

In [1]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

TensorFlow 2.x selected.


### Install dependencies

In [0]:
%%capture
%%bash

pip install -U tensorflow-text

### Import modules

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model

Set default options for modules

In [0]:
pd.set_option('display.max_colwidth', -1)

### GPU check

In [3]:
num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", num_gpus_available)
assert num_gpus_available > 0

Num GPUs Available:  1


In [0]:
config = {
  'seed': 31,
  'batch_size': 64,
  'epochs': 10,
  'max_seq_len': 128
}

## Data

Download the pretrained BERT model

In [5]:
BERT_URL = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
bert_layer = hub.KerasLayer(BERT_URL, trainable=False)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
print(f'BERT vocab is stored at     : {vocab_file}')
print(f'BERT model is case sensitive: {do_lower_case}')

BERT vocab is stored at     : b'/tmp/tfhub_modules/03d6fb3ce1605ad9e5e9ed5346b2fb9623ef4d3d/assets/vocab.txt'
BERT model is case sensitive: True


Load the vocab file that corresponds to the pretrained BERT

In [0]:
def load_vocab(vocab_file):
  """Load a vocabulary file into a list."""
  vocab = []
  with tf.io.gfile.GFile(vocab_file, "r") as reader:
    while True:
      token = reader.readline()
      if not token: break
      token = token.strip()
      vocab.append(token)
  return vocab

vocab = load_vocab(vocab_file)

Use BERT vocab to create a word to index lookup table

In [0]:
def create_vocab_table(vocab, num_oov=1):
  """Create a lookup table for a vocabulary"""
  vocab_values = tf.range(tf.size(vocab, out_type=tf.int64), dtype=tf.int64)
  init = tf.lookup.KeyValueTensorInitializer(keys=vocab, values=vocab_values, key_dtype=tf.string, value_dtype=tf.int64)
  vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov, lookup_key_dtype=tf.string)
  return vocab_table

vocab_lookup_table = create_vocab_table(vocab)

Use BERT vocab to create a index to word lookup table

In [0]:
def create_index2word(vocab):
  # Create a lookup table for a index to token
  vocab_values = tf.range(tf.size(vocab, out_type=tf.int64), dtype=tf.int64)
  init = tf.lookup.KeyValueTensorInitializer(keys=vocab_values, values=vocab)
  return tf.lookup.StaticHashTable(initializer=init, default_value=tf.constant('unk'), name="index2word")

index2word = create_index2word(vocab)

Check out the indices for the following tokens

In [8]:
vocab_lookup_table.lookup(tf.constant(['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]']))

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([  0, 100, 101, 102, 103])>

Check out the token corresponding to an index

In [83]:
index2word.lookup(tf.constant([0], dtype='int64')).numpy()

[b'[PAD]']

Create a BERT tokenizer using TF Text

In [0]:
tokenizer = text.BertTokenizer(
    vocab_lookup_table,
    token_out_type=tf.int64,
    lower_case=do_lower_case
  )

Lookup for the BERT token IDs for padding and start/end of sentence.

In [0]:
PAD_ID = vocab_lookup_table.lookup(tf.constant('[PAD]')) # padding token
CLS_ID = vocab_lookup_table.lookup(tf.constant('[CLS]')) # class token
SEP_ID = vocab_lookup_table.lookup(tf.constant('[SEP]')) # sequence separator token

### Preprocessing

Define the logic to preprocess data and format it as required by BERT

In [0]:
def preprocess(record):
  review, label = record['text'], record['label']
  # process review to calculate BERT input
  ids, mask, type_ids = preprocess_bert_input(review)
  return (ids, mask, type_ids), label

def preprocess_bert_input(review):
  # calculate tokens ID
  ids = tokenize_text(review, config['max_seq_len'])
  # calculate mask
  mask = tf.cast(ids > 0, tf.int64)
  mask = tf.reshape(mask, [-1, config['max_seq_len']])
  # calculate tokens type ID
  zeros_dims = tf.stack(tf.shape(mask))
  type_ids = tf.fill(zeros_dims, 0)
  type_ids = tf.cast(type_ids, tf.int64)

  return (ids, mask, type_ids)

def tokenize_text(review, seq_len):
  # convert text into token ids
  tokens = tokenizer.tokenize(review)
  # flatten the output ragged tensors
  tokens = tokens.merge_dims(1, 2)[:, :seq_len]
  # Add start and end token ids to the id sequence
  start_tokens = tf.fill([tf.shape(review)[0], 1], CLS_ID)
  end_tokens = tf.fill([tf.shape(review)[0], 1], SEP_ID)
  tokens = tokens[:, :seq_len - 2]
  tokens = tf.concat([start_tokens, tokens, end_tokens], axis=1)
  # truncate sequences greater than MAX_SEQ_LEN
  tokens = tokens[:, :seq_len]
  # pad shorter sequences with the pad token id
  tokens = tokens.to_tensor(default_value=PAD_ID)
  pad = seq_len - tf.shape(tokens)[1]
  tokens = tf.pad(tokens, [[0, 0], [0, pad]], constant_values=PAD_ID)

  # and finally reshape the word token ids to fit the output 
  # data structure of TFT 
  return tf.reshape(tokens, [-1, seq_len])

### Dataset

Download the dataset from TF Hub and process it

In [0]:
train_ds, valid_ds = tfds.load('imdb_reviews', split=['train', 'test'], shuffle_files=True)

train_ds = train_ds.shuffle(1024).batch(config['batch_size']).prefetch(tf.data.experimental.AUTOTUNE)
valid_ds = valid_ds.shuffle(1024).batch(config['batch_size']).prefetch(tf.data.experimental.AUTOTUNE)

train_ds, valid_ds = train_ds.map(preprocess), valid_ds.map(preprocess)

## Model

In [0]:
input_ids = Input(shape=(config['max_seq_len'],), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(config['max_seq_len'],), dtype=tf.int32, name="input_mask")
input_type_ids = Input(shape=(config['max_seq_len'],), dtype=tf.int32, name="input_type_ids")

pooled_output, sequence_output = bert_layer([input_ids, input_mask, input_type_ids])
drop_out = Dropout(0.3, name="dropout")(pooled_output)
output = Dense(1, activation='sigmoid', name="linear")(drop_out)

model = Model(inputs=[input_ids, input_mask, input_type_ids], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [35]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_ids[0][0]                  
                                                                 input_mask[0][0]           

### Training

In [36]:
model.fit(train_ds, validation_data=valid_ds, epochs=config['epochs'])

Epoch 1/10
391/391 [==============================] - 499s 1s/step - loss: 0.6654 - accuracy: 0.6016 - val_loss: 0.5977 - val_accuracy: 0.7028
Epoch 2/10
391/391 [==============================] - 510s 1s/step - loss: 0.6063 - accuracy: 0.6712 - val_loss: 0.5650 - val_accuracy: 0.7282
Epoch 3/10
391/391 [==============================] - 510s 1s/step - loss: 0.5839 - accuracy: 0.6969 - val_loss: 0.5494 - val_accuracy: 0.7362
Epoch 4/10
391/391 [==============================] - 511s 1s/step - loss: 0.5730 - accuracy: 0.7025 - val_loss: 0.5388 - val_accuracy: 0.7455
Epoch 5/10
391/391 [==============================] - 510s 1s/step - loss: 0.5696 - accuracy: 0.7058 - val_loss: 0.5376 - val_accuracy: 0.7417
Epoch 6/10
391/391 [==============================] - 510s 1s/step - loss: 0.5613 - accuracy: 0.7146 - val_loss: 0.5268 - val_accuracy: 0.7517
Epoch 7/10
391/391 [==============================] - 510s 1s/step - loss: 0.5608 - accuracy: 0.7130 - val_loss: 0.5233 - val_accuracy: 0.7544

### Evaluation

In [0]:
test_text_ds = tfds.load('imdb_reviews', split='unsupervised', shuffle_files=True)
test_ds = test_text_ds.shuffle(1024).batch(config['batch_size']).prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.map(preprocess)

Check how test text is tokenized

In [0]:
test_text = [record['text'].numpy() for record in test_text_ds.take(10)]

In [0]:
ids = tokenize_text(test_text, config['max_seq_len'])

In [0]:
tokens = [b' '.join(tokens_array) for tokens_array in index2word.lookup(ids).numpy()]

In [91]:
pd.DataFrame({'tokens': tokens})

,tokens
0,"b""[CLS] spoil ##er - now knowing the ending i find it so clever that the whole movie takes place in a motel and each character has a different room . even sane people have many different aspects to their personality , but they don ' t let them become dominant - - they are controlled . malcolm ' s various personalities and needs were person ##ified in each character . the prostitute mother ( amanda pee ##t ) , the part of him who hated her for being a prostitute ( larry ) , the loving mother he wish he had , the loving father he wish he had , the selfish part of himself ( actress ) , the violent part of his personality ( ray [SEP]"""
1,"b""[CLS] i knew about this film long before i saw it . in fact , i had to buy the dvd in order to see it because no video store carried it . i didn ' t mind spending the $ 12 to buy it used because i collect off the wall movies . the new limited edition double dvd has great sound and visually not bad . i found myself laughing much more then < br / > < br / > jolt ##ing in fear , although there were a few scenes were i was startled . < br / > < br / > if you enjoy off the wall 70s sci - fi / horror movies , you probably will eat this one [SEP]"""
2,"b""[CLS] this movie is really really awful . it ' s as bad as zombie 90 well maybe not that bad but pretty close . if your a fan of the italian horror movies then you might like this movie . i thought that it was dam near un ##watch ##able of course i ' m not a fan of the italian movies . the only italian movie that was ok was jungle holocaust . which is one over ##rated movie . this film is way over ##rated . but let ' s get started with how horrible this film really is shall we . the acting is goofy and horrible . the effects suck . no plot with this movie . little gore which is the [SEP]"""
3,"b'[CLS] wait a minute . . . yes i do . < br / > < br / > the director of \' the breed \' has obviously seen terry gill ##iam \' s \' brazil \' a few too many times and asked himself the question , "" if \' brazil \' had been an ill - conceived tale about vampires in the near future , what would it be like ? "" well , i \' ll tell ya , it \' d be like 91 minutes of a swedish whore kicking you in the groin , only not as satisfying . the dialogue was laced with gr ##at ##uit ##ous curse words and tri ##te one - liner ##s , and whoever edited this [SEP]'"
4,"b""[CLS] this is the type of movie that ' s just barely involving enough for one viewing , but i don ' t think i could stand to watch it again . it looks and plays like a mid - seventies tv movie , only with some gr ##at ##uit ##ous sex and violence thrown in . < br / > < br / > i agree with several previous posters - - her ##ve ville ##chai ##ze is not very menacing , and at times even comes off as un ##int ##ended comedy . at least the other two villains make up for that . also , it was jolt ##ing to see jonathan fr ##id is such a pedestrian role , which definitely under - [SEP]"""
5,"b""[CLS] i like sci - fi movies and everything ' bout it and aliens , so i watched this flick . nothing new , nothing special , average acting , typical h . b . davenport ' story , weak and che ##es ##y fx ' s , bad ending of movie , but still the author idea is good . the marines on lost island find the truth about alien landing there and truth about past - experiments on them . they die one after one , some of them were killed by lonely alien , and others by human enemies . ufo effects , when it flees and crush ##es are bad , too . the voices of angry alien are funny , too . [SEP]"""
6,"b""[CLS] i was lucky enough to see a preview of this film tonight . this was a very cool , eerie film . well acted , especially by ska ##rs ##gard who played his role of terry glass perfectly . sob ##ies ##ki did a very good job too as it seems to me that she has a bright future ahead of her . the music was well placed but was fairly standard . the use of shadows was quite interesting as well . overall , this was quite a nice surp

Run prediction on test reviews

In [0]:
result = model.predict(test_ds)

In [38]:
result.shape

(50000, 1)

In [68]:
result_df = pd.DataFrame({'label': tf.squeeze(result[:10]).numpy(), 'text': test_text})
result_df.head()

,label,text
0,0.464566,"b""SPOILER - Now knowing the ending I find it so clever that the whole movie takes place in a motel and each character has a different room. Even sane people have many different aspects to their personality, but they don't let them become dominant -- they are controlled. Malcolm's various personalities and needs were personified in each character. The prostitute mother (Amanda Peet), the part of him who hated her for being a prostitute (Larry), the loving mother he wish he had, the loving father he wish he had, the selfish part of himself (actress), the violent part of his personality (Ray Liotta and Busey), the irrational emotions he feels and his need to be loved (Ginnie) and his attempts to control those feelings (Lou), the hurt little boy who sees far too many traumatic things in his life, and of course, John Cusack who seems to represent Malcolm himself trying to analyze and understand all the craziness in his mind, tries to follow the rules (accepting responsibility for the car accident), help others (giving Amanda Peet a ride, and stitching up the mother). Very cleverly done!"""
1,0.252326,"b'I knew about this film long before I saw it. In fact, I had to buy the DVD in order to see it because no video store carried it. I didn\'t mind spending the $12 to buy it used because I collect off the wall movies. The new limited edition double DVD has great sound and visually not bad. I found myself laughing much more then<br /><br />jolting in fear, although there were a few scenes were I was startled.<br /><br />If you enjoy off the wall 70s sci-fi/horror movies, you probably will eat this one up. I was a little dissapointed at how abrubtly it ended. I wanted the movie to keep going, see how things pan out. The DVD revolution has brought so many<br /><br />lost clasics back to life, it is truly wonderful. Blue Sunshine is one of those lost ""missing links"" of the cinema. Enjoy!'"
2,0.485239,"b""This movie is really really awful. It's as bad as Zombie 90 well maybe not that bad but pretty close. If your a fan of the Italian horror movies then you might like this movie. I thought that it was dam near unwatchable of course I'm not a fan of the Italian movies. The only Italian movie that was OK was Jungle holocaust. Which is one overrated movie. This film is way overrated. But let's get started with how horrible this film really is shall we. The acting is goofy and horrible. The effects suck. No plot with this movie. Little gore which is the only good thing in the film isn't showed nearly enough to be worth watching this wreck. The zombies are very fake looking. It looks like it's a bunch of dudes wearing cheap dollar store masks. Please avoid this film at all costs."""
3,0.251897,"b'Wait a minute... yes I do.<br /><br />The director of \'The Breed\' has obviously seen Terry Gilliam\'s \'Brazil\' a few too many times and asked himself the question, ""If \'Brazil\' had been an ill-conceived tale about vampires in the near future, what would it be like?"" Well, I\'ll tell ya, it\'d be like 91 minutes of a Swedish whore kicking you in the groin, only not as satisfying. The dialogue was laced with gratuitous curse words and trite one-liners, and whoever edited this piece of crap should be shot. I have no real idea of exactly how the whole thing ended because I\'m not really sure what happened during the first part of the film. With so many subplots your head begins to hurt and so much bad acting your head wants to explode this movie should only be viewed with large quantities of beer and at least two other people you can MST3K with. The only thing that made me not stab myself in the eye with a dirty soup spoon was this line: Evil Doctor Guy: ""That\'s it, you are not James Bond, and I am not Blofeld. No more explanations!"" Dude From Jason\'s Lyric: ""I\'m getting paid scale!"" The cinematography was shaky at best and the acting was putrid. Also, what was with all the pseudo-1984 posters and PA announcements? T